In [1]:
import tensorflow

C:\Users\shaha\anaconda3\envs\tensor\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\shaha\anaconda3\envs\tensor\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\shaha\anaconda3\envs\tensor\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:\Users\shaha\anaconda3\envs\tensor\lib\site-packages\tensorflow\pyth

In [2]:
import tensorflow
from flask import Flask, request, jsonify
from flask_cors import CORS
import cv2
import numpy as np
from PIL import Image
from tensorflow import keras
import os
from mtcnn.mtcnn import MTCNN
from numpy import asarray

In [3]:
from tensorflow.keras.models import model_from_json

In [4]:
import io
import matplotlib.pyplot as plt

In [5]:
mtcnn_detector = MTCNN()

Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


In [6]:
# Function to detect and extract face from a image

def detect_face(filename, required_size=(160, 160),normalize = True):
    img = Image.open(filename)
    # convert to RGB
    img = img.convert('RGB')
    # convert to array
    pixels = np.array(img)
    print("pixels successfully")
    
    # detect faces in the image
    results = mtcnn_detector.detect_faces(pixels)
    print("results successfully") 
    if(len(results)==0):
        return None

    # extract the bounding box from the first face
    x1, y1, width, height = results[0]['box']

    x1, y1 = abs(x1), abs(y1)
    x2, y2 = x1 + width, y1 + height
    
    print("extract successfully")  
    # extract the face
    face = pixels[y1:y2, x1:x2]
  
    # resize pixels to the model size
    image = Image.fromarray(face)
    image = image.resize(required_size)
    face_array = asarray(image)
 
    if normalize == True:

        mean = np.mean(face_array, axis=(0,1,2), keepdims=True)
        std = np.std(face_array, axis=(0,1,2), keepdims=True)
        std_adj = np.maximum(std, 1.0)
        return (face_array - mean) / std

    else : 
        return face_array

In [7]:
# Load pretrained Inception-ResNet-v1 model
# Update model and weights path according to your working environment

model_path = "Models/Inception_ResNet_v1.json"
weights_path = "Models/enc_model_weights.h5"

json_file = open(model_path, 'r')
loaded_model_json = json_file.read()
json_file.close()
enc_model = model_from_json(loaded_model_json)
enc_model.load_weights(weights_path)

In [8]:
enc_model.summary()

Model: "inception_resnet_v1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 160, 160, 3) 0                                            
__________________________________________________________________________________________________
Conv2d_1a_3x3 (Conv2D)          (None, 79, 79, 32)   864         input_1[0][0]                    
__________________________________________________________________________________________________
Conv2d_1a_3x3_BatchNorm (BatchN (None, 79, 79, 32)   96          Conv2d_1a_3x3[0][0]              
__________________________________________________________________________________________________
Conv2d_1a_3x3_Activation (Activ (None, 79, 79, 32)   0           Conv2d_1a_3x3_BatchNorm[0][0]    
________________________________________________________________________________

In [9]:
# Compute Face encodings and load IDs of known persons
# Update face database path according to your working environment

known_faces_encodings = []
known_faces_ids = []

known_faces_path = "Face_database/"

for filename in os.listdir(known_faces_path):
  
  # Detect faces
  face = detect_face(known_faces_path+filename,normalize = True)

  # Compute face encodings

  feature_vector = enc_model.predict(face.reshape(1,160,160,3))
  feature_vector/= np.sqrt(np.sum(feature_vector**2))
  known_faces_encodings.append(feature_vector)

  # Save Person IDs
  label = filename.split('.')[0]
  known_faces_ids.append(label)


known_faces_encodings = np.array(known_faces_encodings).reshape(len(known_faces_encodings),128)
known_faces_ids = np.array(known_faces_ids)

pixels successfully
results successfully
extract successfully
pixels successfully
results successfully
extract successfully
pixels successfully
results successfully
extract successfully


In [10]:
# Function to recognize a face (if it is in known_faces)

def recognize(img,known_faces_encodings,known_faces_ids,threshold = 0.85):

  scores = np.zeros((len(known_faces_ids),1),dtype=float)

  enc = enc_model.predict(img.reshape(1,160,160,3))
  enc/= np.sqrt(np.sum(enc**2))

  scores = np.sqrt(np.sum((enc-known_faces_encodings)**2,axis=1))

  match = np.argmin(scores)

  if scores[match] > threshold :
    print("scores", scores)
    return ("UNKNOWN",0)
      
  else :

    return (known_faces_ids[match],scores[match])

In [11]:
from flask import Flask, request, jsonify
from flask_cors import CORS
import io
from PIL import Image, ImageOps
import numpy as np
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
from tensorflow.keras.models import load_model, model_from_json
from numpy import asarray

app = Flask(__name__)
app.secret_key = 'MAHYA_APP_13579@MAHYA'
CORS(app)

@app.route("/", methods=["GET", "POST"])
def recognize_face():
    if "image" not in request.files:
        return jsonify({"error": "No image file found in the request"}), 400

    img_bytes = request.files["image"].read()
    img_pil = Image.open(io.BytesIO(img_bytes))
    mtcnn_detector = MTCNN()

    # List to hold results from each rotation
    all_results = []

    # Rotate image and apply detector
    for i in range(0,4):
        # Rotate the image by 90 degrees * i times
        rotated_image = img_pil.rotate(90 * i)

        # Convert to numpy array and apply face detection
        pixels = np.array(rotated_image)
        results = mtcnn_detector.detect_faces(pixels)
        
        if len(results) == 0:
            # If no faces are detected in this rotation
            continue  # Move to the next rotation

        # If faces are detected, process them
        for result in results:
            # Extract the bounding box
            x1, y1, width, height = result['box']
            x1, y1 = abs(x1), abs(y1)
            x2, y2 = x1 + width, y1 + height

            # Extract the face and resize it
            face = pixels[y1:y2, x1:x2]
            face_image = Image.fromarray(face).resize((160,160))
            face_array = asarray(face_image)

            # Normalize the face data
            mean = np.mean(face_array, axis=(0,1,2), keepdims=True)
            std = np.std(face_array, axis=(0,1,2), keepdims=True)
            std_adj = np.maximum(std, 1.0)
            normalized_face = (face_array - mean) / std_adj

            # Recognize the face
            label, score = recognize(normalized_face, known_faces_encodings, known_faces_ids, threshold=0.95)
            all_results.append({"rotation": 90 * i, "label": label})

    # If no faces were detected in any rotation, return an error message
    if len(results) == 0:
        return jsonify({"error": "No faces detected"}), 404

    # Determine which label to return
    known_labels = [result["label"] for result in all_results if result["label"].upper() != "UNKNOWN"]

    if known_labels:
        # If there are known labels, return the first one
        return jsonify({"label": known_labels[0]})
    else:
        # If all labels are "UNKNOWN", return "UNKNOWN"
        return jsonify({"label": "UNKNOWN"})

@app.errorhandler(500)
def handle_internal_error(error):
    # Log the error or perform other actions
    error_response = {
        "error": "Internal Server Error",
        "message": str(error),
    }
    return jsonify(error_response), 500

if __name__ == "__main__":
    app.run(host="127.0.0.1", port=7777, debug=False, threaded=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:7777
Press CTRL+C to quit
